# Mapbox Stitcher

This notebook contains tools to fetch tiles from mapbox, in a user-defined style, and stitch them together into 1 big map image

### Usage

1) Add a line in the file "regions.csv" like the examples with the parameters you want. For the correspondence between zoomlevel and scale, see: https://wiki.openstreetmap.org/wiki/Zoom_levels

2) fill in your mapbox acces token and username (username is only needed for custom styles)

3) in REGION_NAME: fille in the name you have given the region in the csv file

4) go to the section "Print out image info", klick on the code cell. Then click in the topbar on "run", click "run all obove selected cell", hit ctrl+enter. This wil print out information about the map to be generated. your available RAM/10 is a save upper bound for the size of the map that will be generated.

5) If the info is as expected, go to "run"->"run selected cell and below". This will fetch the tiles from the server and construct the map. Large maps can take while to download and stitch together!

! watch out for warnings about tilesize, a rong tilesize leads to wrong metadata and coordinate transformations

Timing: generating a 27x18 tiles image, corresponding to 13824x9216 pixels or 127.40 Mpixel, takes about a minute with decent internet on a intel 7th gen i5 laptop. 

Memory usage: ~1.5GB

Size output image: 54.1MB

## Imports

In [1]:
# numpy for array manipulations
import numpy as np
# the workhorse, provides routines to download tiles and do some
# coordinate transforms
import cartopy.io.img_tiles as cartotiles
# image manipulating and saving
from PIL import Image
# more coordinate support
#import globalmaptiles as gmaptiles
# for working with metadata in csv
import pandas as pd

# custom functions
import tilemaptools as tmt

## Global variables
This should be the only variables you need to change to generate a different map, unless you need more in-depth modifications. Changes to these maps should be handled as much as possible by changing the mapbox style, or doing postprocessing on the resulting image

In [1]:
# acces token for mapbox
TOKEN = "FILL IN ACCES TOKEN"
# username corresponding to token, only needed for private styles
USER = "FILL IN USERNAME"

# descriptive name for region of interest, used to fetch map info from regions.csv
REGION_NAME = "Belgie"

# if you intend to print the map, set the DPI value to get the physical size of the image
DPI = 200
# size in pixels of 1 tile. Default is 256 unless you did "louche aanpassing" in the sourcecode of cartopy
TILESIZE = 512

# you shouldn't need to change these ;)
CM_PER_INCH = 2.54
EARTH_CIRCUMFERENCE = 40_075_016.686   # meter
SEP = ","

### Import map data from regions.csv

In [10]:
regionfile = pd.read_csv("regions.csv", sep=SEP)
regionfile = regionfile.set_index("Region")

MAP_PARAMS = regionfile.loc[REGION_NAME]
ZOOM = int(MAP_PARAMS["ZoomLevel"])
LON = (MAP_PARAMS["LonMin"], MAP_PARAMS["LonMax"])
LAT = (MAP_PARAMS["LatMin"], MAP_PARAMS["LatMax"])
STYLE_NAME = MAP_PARAMS["StyleName"]
STYLE_ID = MAP_PARAMS["StyleID"]
CUSTOM_TILESET = MAP_PARAMS["CustomTileset"]
FOLDER = MAP_PARAMS["Folder"]

# instance to download tiles
if CUSTOM_TILESET:
    maptiles = cartotiles.MapboxStyleTiles(TOKEN, USER, STYLE_ID, cache=False)
else:
    maptiles = cartotiles.MapboxTiles(TOKEN, STYLE_ID)

## calculate extra map metadata from user input and print image description

In [12]:
meta = tmt.calculate_metadata(LAT, LON, ZOOM, maptiles, TILESIZE, DPI=DPI)
print(meta["Description"])

------ SIZE ------

Size of resulting image will be (in pixels): 
    width: 12288
    height: 9728
    yielding a 119.54 Megapixel image

When printed in a resolution of 200.0 DPI, the physical size of this print in cm is:
    width: 156.06
    height: 123.55
    yielding a 1.928 m^2 print

The scale of the map in meter/pixel is:
    top: 23.789 
    bottom: 24.823

-----COVERAGE-------

The actual geographical area covered ranges is between:
    lat: 49.382373° - 51.508742°
    lon: 2.460937° - 6.679687°
This leads to a map that is 11.11% bigger than the region of interest


## download and save the image

In [13]:
# download the tiles
tiles = tmt.images_for_domain(maptiles, meta["AreaPolygon"], ZOOM)

# fix tilesize in case it was wrong
actual_tilesize = np.shape(tiles[0][0])[0]
if actual_tilesize != TILESIZE:
    print("WARNING: TILESIZE defined in General Parameters does not match tilesize recieved from server.")
    print("actual tilesize is: %d"%actual_tilesize)
    print("consider running the notebook again with the right tilesize to generate correct metadata")
    TILESIZE = actual_tilesize

In [14]:
img = tmt.merge_tiles(tiles, meta["PixelSize"], meta["AreaTiles"], TILESIZE)

In [15]:
PIL_image = Image.fromarray(img)
PIL_image.save("%s/%s_%s_%s.png"%(FOLDER, REGION_NAME, STYLE_NAME, ZOOM), "png")

## write map metadata to csv

Necessary to create other layers for map like a heatlayer

In [16]:
COLUMNS = ["MapName", "Region", "StyleName", "StyleID", "IsPrivateStyle", "user", 
           "LatMin", "LatMax", "LonMin", "LonMax", "Zoom", "TileSize", 
           "PixWidth", "PixHeight", "PixScaleTop", "PixScaleBottom",
           "MeterXMin", "MeterXMax", "MeterYMin", "MeterYMax",
           "MapLatMin", "MapLatMax", "MapLonMin", "MapLonMax",
           "MapPixXMin", "MapPixXMax", "MapPixYMin", "MapPixYMax",
           "SurplusArea"]
MAPNAME = "%s_%s_%s"%(REGION_NAME, STYLE_NAME, ZOOM)

# try to open the csv in case it already exists
try: 
    df = pd.read_csv("maps_metadata.csv", sep=SEP)
except:
    # csv doesn't exist, create new dataframe
    df = pd.DataFrame(columns=COLUMNS)
df = df.set_index("MapName")

# check if this map is already in the csv
if MAPNAME in df.index.values:
    # drop the row
    df = df.drop(MAPNAME)

# create data array
data = {"MapName": MAPNAME,
        "Region" : REGION_NAME, 
        "StyleName" : STYLE_NAME, 
        "StyleID" : STYLE_ID, 
        "IsPrivateStyle" : CUSTOM_TILESET, 
        "user" : USER, 
        "LatMin" : LAT[0], 
        "LatMax" : LAT[1], 
        "LonMin" : LON[0], 
        "LonMax" : LON[1], 
        "Zoom" : ZOOM, 
        "TileSize" : TILESIZE, 
        "PixWidth" : meta["PixelSize"][0], 
        "PixHeight" : meta["PixelSize"][1], 
        "PixScaleTop" : meta["ScaleTop"], 
        "PixScaleBottom" : meta["ScaleBottom"],
        "MeterXMin" : meta["PseudoMercatorExtent"][0], 
        "MeterXMax" : meta["PseudoMercatorExtent"][1], 
        "MeterYMin" : meta["PseudoMercatorExtent"][2], 
        "MeterYMax" : meta["PseudoMercatorExtent"][3],
        "MapLatMin" : meta["LatLonExtent"][0], 
        "MapLatMax" : meta["LatLonExtent"][1], 
        "MapLonMin" : meta["LatLonExtent"][2], 
        "MapLonMax" : meta["LatLonExtent"][3],
        "MapPixXMin" : meta["PixelExtent"][0], 
        "MapPixXMax" : meta["PixelExtent"][1], 
        "MapPixYMin" : meta["PixelExtent"][2], 
        "MapPixYMax" : meta["PixelExtent"][3],
        "SurplusArea" : meta["SurplusArea"]}#, 
        #"description" : description}

# append data array
#df = df.append(data, ignore_index=True)
df.loc[data["MapName"]] = [data[key] for key in COLUMNS[1:]]

# write out csv
df.to_csv("maps_metadata.csv", sep=SEP)